In [1]:
%%time

display("==== Starting 0-1 Image Acquisition ====")

import datetime as dt
import os
from datetime import datetime
import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm

from common.captioning.caption import BlipCaption
from common.schemas.pyarrow_schema import schema
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.functions.functions import Functions

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

'==== Starting 0-1 Image Acquisition ===='

CPU times: total: 2.41 s
Wall time: 2.45 s


In [2]:
%%time

# "memes","CityPorn", "EarthPorn", "spaceporn",	"itookapicture", "trippinthroughtime"
subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"fatsquirrelhate",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH",
	"bathandbodyworks",
	"AesPleasingAsianGirls"
]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion","AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners" "hotofficegirls", "tightdresses", "DLAH"]},{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time

extant_data = pandas.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True)
display(extant_data)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]


CPU times: total: 266 ms
Wall time: 882 ms


In [5]:
%%time

curated_data = pandas.read_parquet("data/parquet/primary_caption.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
curated_data.set_index('id', inplace=True)

display("==== CURATED DATA ====")
display(curated_data.shape)
display(curated_data)

'==== CURATED DATA ===='

(39791, 14)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13nwjqp,DLAH,spicy_kingWest44,Gotta get them angles,two women in short dresses standing next to a ...,26a955b5d75d32d39ac452a9a934d15d,/r/DLAH/comments/13nwjqp/gotta_get_them_angles/,https://i.redd.it/7xdtvt84l81b1.jpg,13nwjqp.jpg,data/image/13nwjqp.jpg,PrettyGirlDiffusion,True,True,False,[]
13nx3q5,DLAH,robok212,I know there is not much dress ... but there i...,araffe woman in a black lingerie posing in a d...,5fd63ab2fb5a39229ba0eba3a2775f8a,/r/DLAH/comments/13nx3q5/i_know_there_is_not_m...,https://i.redd.it/1jdres3dp81b1.jpg,13nx3q5.jpg,data/image/13nx3q5.jpg,PrettyGirlDiffusion,True,True,False,[]
13nt4ei,bathandbodyworks,LadyMisfits,Nice surprise from Micheal’s,someone holding a jar of candles in front of a...,362b6a403ebe68409fdefbcf7729ffbc,/r/bathandbodyworks/comments/13nt4ei/nice_surp...,https://i.redd.it/mse5auo4681b1.jpg,13nt4ei.jpg,data/image/13nt4ei.jpg,CandleDiffusion,True,True,True,[]


CPU times: total: 297 ms
Wall time: 644 ms


In [6]:
%%time

extant_data.update(curated_data)
updated_extant_data = extant_data.reset_index()

display("==== UPDATED EXTANT DATA ====")
display(updated_extant_data.shape)
display(updated_extant_data)
updated_extant_data.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

'==== UPDATED EXTANT DATA ===='

(41505, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41500,131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
41501,131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
41502,131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]
41503,131jvcx,DLAH,robok212,Good morning 😍,araffe in a pink dress posing on a white wall,01f5058855526ef25888cff88065db4c,/r/DLAH/comments/131jvcx/good_morning/,https://i.redd.it/ozmbfhjv3mwa1.jpg,131jvcx.jpg,data/image/131jvcx.jpg,PrettyGirlDiffusion,True,True,False,[]


CPU times: total: 250 ms
Wall time: 2.71 s


In [7]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

CPU times: total: 31.2 ms
Wall time: 105 ms


In [8]:
for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 18/18 [00:00<00:00, 8873.70it/s]


In [9]:
with open('log.txt', 'a') as f:
	for time_filter in ['week', 'day']:
		s = tqdm(subs, desc=f"{time_filter}", total=len(subs))
		for sub in s:
		# for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
			temp_dir_path = f"temp/{sub}"
			try:
				subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
				subreddit_stream = list(subreddit_stream)
				p = tqdm(subreddit_stream, total=len(subreddit_stream), desc=f"Posts - {sub} - {time_filter}")
				for submission in p:
					if submission is None:
						continue
					else:
						if submission.id in extant_data.index.values:
							continue
					try:
						author_name = 'Unknown'
						subreddit_name = sub
						try:
							author_name = submission.author.name
						except Exception as e:
							author_name = 'Unknown'
							pass
						p = {
							'id': submission.id,
							'subreddit': subreddit_name,
							'author': author_name,
							'title': submission.title,
							'caption': '',
							'hash': '',
							'permalink': submission.permalink,
							'original_url': submission.url,
							'image_name': '',
							'path': '',
							'thumbnail_path': '',
							'exists': False,
							'curated': False,
							'Tags': ['']
						}
						pd.DataFrame([p]).to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
					except Exception as e:
						log = functions.write_log_message(submission.id, sub, "Error Writing Post", e)
						f.write(log)
						continue
			except Exception as e:
				log = functions.write_log_message(submission.id, sub, f"Error Getting Posts For SubReddit", e)
				f.write(log)
				continue

day: 100%|██████████| 18/18 [00:47<00:00,  2.62s/it]


In [10]:
%%time

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data)

Reading Temp Dir For Subs...: 100%|██████████| 18/18 [00:01<00:00, 13.60it/s]


'==== TEMP DATA ===='

(3266, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13h67sb,13h67sb,SFWRedheads,Dasharkiss_real,Love life and life will love you too.,,,/r/SFWRedheads/comments/13h67sb/love_life_and_...,https://i.imgur.com/NBCDz3f.jpg,13h67sb.jpg,,,False,False,False,[]
13h93zj,13h93zj,SFWRedheads,thatgirlmarie97,Who wants to play some games?,,,/r/SFWRedheads/comments/13h93zj/who_wants_to_p...,https://www.reddit.com/gallery/13h93zj,13h93zj.jpg,,,False,False,False,[]
13h9lq5,13h9lq5,SFWRedheads,snildeep,Hey! My eyes are up here! 😂💕,,,/r/SFWRedheads/comments/13h9lq5/hey_my_eyes_ar...,https://i.redd.it/schip1qaotza1.jpg,13h9lq5.jpg,,,False,False,False,[]
13hc2uz,13hc2uz,SFWRedheads,nurselacey,How do you like my haircut?,,,/r/SFWRedheads/comments/13hc2uz/how_do_you_lik...,https://i.redd.it/0qgvz4uy9uza1.jpg,13hc2uz.jpg,,,False,False,False,[]
13hcvxr,13hcvxr,SFWRedheads,Puzzled_MJ,Happy Sunday. Where would u take me today?,,,/r/SFWRedheads/comments/13hcvxr/happy_sunday_w...,https://i.redd.it/h6bwkyw8guza1.jpg,13hcvxr.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13qlj64,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,,,False,False,False,[]
13qmsn4,13qmsn4,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,,/r/AesPleasingAsianGirls/comments/13qmsn4/song...,https://i.redd.it/cdj3scvnvt1b1.jpg,13qmsn4.jpg,,,False,False,False,[]
13r71qv,13r71qv,AesPleasingAsianGirls,Defiant-Appearance64,RedVelvet Yeri,,,/r/AesPleasingAsianGirls/comments/13r71qv/redv...,https://i.imgur.com/hEcFHmc.gifv,13r71qv.jpg,,,False,False,False,[]


CPU times: total: 10.2 s
Wall time: 4.16 s


In [11]:
%%time

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & temp_data['original_url'].str.endswith('.jpg') & (~temp_data['id'].isin(curated_data.index.values))]

filtered.dropna(axis=1, how='all')

filtered.reset_index(drop=True, inplace=True)

filtered.set_index('id', inplace=True, drop=False)

display("==== FILTERED DATA ====")

display(filtered.shape)

display(filtered)

'==== FILTERED DATA ===='

(837, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,,False,False,False,[]
13o2vke,13o2vke,SFWRedheads,nurselacey,Sunday Funday,,,/r/SFWRedheads/comments/13o2vke/sunday_funday/,https://i.redd.it/5c4jh6kmu91b1.jpg,13o2vke.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13po3eq,13po3eq,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13po3eq/habin/,https://i.redd.it/6an1c1zagm1b1.jpg,13po3eq.jpg,,,False,False,False,[]
13pq23t,13pq23t,AesPleasingAsianGirls,theowuriwufoa,Car selfie,,,/r/AesPleasingAsianGirls/comments/13pq23t/car_...,https://i.redd.it/3lg7bn3ntm1b1.jpg,13pq23t.jpg,,,False,False,False,[]
13qlj64,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,,,False,False,False,[]


CPU times: total: 15.6 ms
Wall time: 22.5 ms


In [12]:
%%time

filtered['model'] = filtered.progress_apply(lambda x: functions.add_source(x, sources), axis=1)

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 837/837 [00:00<00:00, 118446.39it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Model =='

(837, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13o2vke,13o2vke,SFWRedheads,nurselacey,Sunday Funday,,,/r/SFWRedheads/comments/13o2vke/sunday_funday/,https://i.redd.it/5c4jh6kmu91b1.jpg,13o2vke.jpg,,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13po3eq,13po3eq,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13po3eq/habin/,https://i.redd.it/6an1c1zagm1b1.jpg,13po3eq.jpg,,SexyAsianDiffusion,False,False,False,[]
13pq23t,13pq23t,AesPleasingAsianGirls,theowuriwufoa,Car selfie,,,/r/AesPleasingAsianGirls/comments/13pq23t/car_...,https://i.redd.it/3lg7bn3ntm1b1.jpg,13pq23t.jpg,,SexyAsianDiffusion,False,False,False,[]
13qlj64,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,,SexyAsianDiffusion,False,False,False,[]


CPU times: total: 31.2 ms
Wall time: 25.2 ms


In [13]:
%%time

file_list = file_system.ls("data/image")

filtered['path'] = filtered.progress_apply(lambda x: functions.fetch_image(x, file_list, file_system), axis=1)

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 837/837 [07:00<00:00,  1.99it/s]
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Path =='

(837, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13o2vke,13o2vke,SFWRedheads,nurselacey,Sunday Funday,,,/r/SFWRedheads/comments/13o2vke/sunday_funday/,https://i.redd.it/5c4jh6kmu91b1.jpg,13o2vke.jpg,data/image/13o2vke.jpg,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13po3eq,13po3eq,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13po3eq/habin/,https://i.redd.it/6an1c1zagm1b1.jpg,13po3eq.jpg,data/image/13po3eq.jpg,SexyAsianDiffusion,False,False,False,[]
13pq23t,13pq23t,AesPleasingAsianGirls,theowuriwufoa,Car selfie,,,/r/AesPleasingAsianGirls/comments/13pq23t/car_...,https://i.redd.it/3lg7bn3ntm1b1.jpg,13pq23t.jpg,data/image/13pq23t.jpg,SexyAsianDiffusion,False,False,False,[]
13qlj64,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,data/image/13qlj64.jpg,SexyAsianDiffusion,False,False,False,[]


CPU times: total: 1min 22s
Wall time: 7min 28s


In [14]:
%%time

filtered['exists'] = filtered.progress_apply(lambda x: functions.set_exists(x), axis=1)

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 837/837 [00:00<00:00, 26215.77it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Exists =='

(837, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13o2vke,13o2vke,SFWRedheads,nurselacey,Sunday Funday,,,/r/SFWRedheads/comments/13o2vke/sunday_funday/,https://i.redd.it/5c4jh6kmu91b1.jpg,13o2vke.jpg,data/image/13o2vke.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13po3eq,13po3eq,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13po3eq/habin/,https://i.redd.it/6an1c1zagm1b1.jpg,13po3eq.jpg,data/image/13po3eq.jpg,SexyAsianDiffusion,True,False,False,[]
13pq23t,13pq23t,AesPleasingAsianGirls,theowuriwufoa,Car selfie,,,/r/AesPleasingAsianGirls/comments/13pq23t/car_...,https://i.redd.it/3lg7bn3ntm1b1.jpg,13pq23t.jpg,data/image/13pq23t.jpg,SexyAsianDiffusion,True,False,False,[]
13qlj64,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,data/image/13qlj64.jpg,SexyAsianDiffusion,True,False,False,[]


CPU times: total: 46.9 ms
Wall time: 48.2 ms


In [15]:
%%time

filtered['hash'] = filtered.progress_apply(lambda x: functions.set_hash(x), axis=1)

display("== Filtered Data With Hash ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 837/837 [00:00<00:00, 1122.11it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Hash =='

(837, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13o2vke,13o2vke,SFWRedheads,nurselacey,Sunday Funday,,0fae4d9dbb466999e34ceb2a8581f136,/r/SFWRedheads/comments/13o2vke/sunday_funday/,https://i.redd.it/5c4jh6kmu91b1.jpg,13o2vke.jpg,data/image/13o2vke.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13po3eq,13po3eq,AesPleasingAsianGirls,throwaway83736365372,Habin,,3ea79e0ffa26050402bd62a47bf15000,/r/AesPleasingAsianGirls/comments/13po3eq/habin/,https://i.redd.it/6an1c1zagm1b1.jpg,13po3eq.jpg,data/image/13po3eq.jpg,SexyAsianDiffusion,True,False,False,[]
13pq23t,13pq23t,AesPleasingAsianGirls,theowuriwufoa,Car selfie,,4b03667021cb1a4694dc982e21c0f7c1,/r/AesPleasingAsianGirls/comments/13pq23t/car_...,https://i.redd.it/3lg7bn3ntm1b1.jpg,13pq23t.jpg,data/image/13pq23t.jpg,SexyAsianDiffusion,True,False,False,[]
13qlj64,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,e1244a45a5018d6d35e487217a489db2,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,data/image/13qlj64.jpg,SexyAsianDiffusion,True,False,False,[]


CPU times: total: 781 ms
Wall time: 763 ms


In [16]:
ts = datetime.today().timestamp()

exists = filtered.loc[filtered['exists'] == True]
exists.dropna(axis=1, how='all')
exists.reset_index(drop=True, inplace=True)

new_records_to_process = exists.copy()
del exists

new_records_to_process.reset_index(drop=True, inplace=True)
display(new_records_to_process)

out_name = f"data/temp/caption/primary/{ts}_new_records_to_process.parquet"
new_records_to_process.to_parquet(out_name, engine='pyarrow', filesystem=file_system, schema=schema)
del new_records_to_process

new_df = pandas.read_parquet(out_name, engine='pyarrow', filesystem=file_system, schema=schema)

display("== New Records To Process ==")
display(new_df.shape)
display(new_df)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,13o2vke,SFWRedheads,nurselacey,Sunday Funday,,0fae4d9dbb466999e34ceb2a8581f136,/r/SFWRedheads/comments/13o2vke/sunday_funday/,https://i.redd.it/5c4jh6kmu91b1.jpg,13o2vke.jpg,data/image/13o2vke.jpg,RedHeadDiffusion,True,False,False,[]
1,13o36lc,SFWRedheads,RedStringGirl,I Hope You All Having A Good Sunday,,e84ee51461ed2ea3c4deb7c3991b880a,/r/SFWRedheads/comments/13o36lc/i_hope_you_all...,https://i.redd.it/y684k1tvw91b1.jpg,13o36lc.jpg,data/image/13o36lc.jpg,RedHeadDiffusion,True,False,False,[]
2,13o37bb,SFWRedheads,Ms_britt_one,Happy Sunday everyone! 😘,,bf023e7dbc4f77175e126784667660fe,/r/SFWRedheads/comments/13o37bb/happy_sunday_e...,https://i.redd.it/gomgimf0x91b1.jpg,13o37bb.jpg,data/image/13o37bb.jpg,RedHeadDiffusion,True,False,False,[]
3,13o4j6c,SFWRedheads,Bakedredhead,"Don’t normally wear makeup, how did I do??",,9913d43bd109cb476bf5309159eb9c70,/r/SFWRedheads/comments/13o4j6c/dont_normally_...,https://i.redd.it/qyjw35kk6a1b1.jpg,13o4j6c.jpg,data/image/13o4j6c.jpg,RedHeadDiffusion,True,False,False,[]
4,13o55kx,SFWRedheads,Ovaxx25,Went to see my colleagues before I leave this ...,,6bc35b942fc303ee427929f36d324a10,/r/SFWRedheads/comments/13o55kx/went_to_see_my...,https://i.redd.it/wf9gqcb5ba1b1.jpg,13o55kx.jpg,data/image/13o55kx.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,13po3eq,AesPleasingAsianGirls,throwaway83736365372,Habin,,3ea79e0ffa26050402bd62a47bf15000,/r/AesPleasingAsianGirls/comments/13po3eq/habin/,https://i.redd.it/6an1c1zagm1b1.jpg,13po3eq.jpg,data/image/13po3eq.jpg,SexyAsianDiffusion,True,False,False,[]
818,13pq23t,AesPleasingAsianGirls,theowuriwufoa,Car selfie,,4b03667021cb1a4694dc982e21c0f7c1,/r/AesPleasingAsianGirls/comments/13pq23t/car_...,https://i.redd.it/3lg7bn3ntm1b1.jpg,13pq23t.jpg,data/image/13pq23t.jpg,SexyAsianDiffusion,True,False,False,[]
819,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,e1244a45a5018d6d35e487217a489db2,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,data/image/13qlj64.jpg,SexyAsianDiffusion,True,False,False,[]
820,13qmsn4,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,a2189d7026d58082336ddcad3667161e,/r/AesPleasingAsianGirls/comments/13qmsn4/song...,https://i.redd.it/cdj3scvnvt1b1.jpg,13qmsn4.jpg,data/image/13qmsn4.jpg,SexyAsianDiffusion,True,False,False,[]


'== New Records To Process =='

(822, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,13o2vke,SFWRedheads,nurselacey,Sunday Funday,,0fae4d9dbb466999e34ceb2a8581f136,/r/SFWRedheads/comments/13o2vke/sunday_funday/,https://i.redd.it/5c4jh6kmu91b1.jpg,13o2vke.jpg,data/image/13o2vke.jpg,RedHeadDiffusion,True,False,False,[]
1,13o36lc,SFWRedheads,RedStringGirl,I Hope You All Having A Good Sunday,,e84ee51461ed2ea3c4deb7c3991b880a,/r/SFWRedheads/comments/13o36lc/i_hope_you_all...,https://i.redd.it/y684k1tvw91b1.jpg,13o36lc.jpg,data/image/13o36lc.jpg,RedHeadDiffusion,True,False,False,[]
2,13o37bb,SFWRedheads,Ms_britt_one,Happy Sunday everyone! 😘,,bf023e7dbc4f77175e126784667660fe,/r/SFWRedheads/comments/13o37bb/happy_sunday_e...,https://i.redd.it/gomgimf0x91b1.jpg,13o37bb.jpg,data/image/13o37bb.jpg,RedHeadDiffusion,True,False,False,[]
3,13o4j6c,SFWRedheads,Bakedredhead,"Don’t normally wear makeup, how did I do??",,9913d43bd109cb476bf5309159eb9c70,/r/SFWRedheads/comments/13o4j6c/dont_normally_...,https://i.redd.it/qyjw35kk6a1b1.jpg,13o4j6c.jpg,data/image/13o4j6c.jpg,RedHeadDiffusion,True,False,False,[]
4,13o55kx,SFWRedheads,Ovaxx25,Went to see my colleagues before I leave this ...,,6bc35b942fc303ee427929f36d324a10,/r/SFWRedheads/comments/13o55kx/went_to_see_my...,https://i.redd.it/wf9gqcb5ba1b1.jpg,13o55kx.jpg,data/image/13o55kx.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,13po3eq,AesPleasingAsianGirls,throwaway83736365372,Habin,,3ea79e0ffa26050402bd62a47bf15000,/r/AesPleasingAsianGirls/comments/13po3eq/habin/,https://i.redd.it/6an1c1zagm1b1.jpg,13po3eq.jpg,data/image/13po3eq.jpg,SexyAsianDiffusion,True,False,False,[]
818,13pq23t,AesPleasingAsianGirls,theowuriwufoa,Car selfie,,4b03667021cb1a4694dc982e21c0f7c1,/r/AesPleasingAsianGirls/comments/13pq23t/car_...,https://i.redd.it/3lg7bn3ntm1b1.jpg,13pq23t.jpg,data/image/13pq23t.jpg,SexyAsianDiffusion,True,False,False,[]
819,13qlj64,AesPleasingAsianGirls,throwaway83736365372,Habin,,e1244a45a5018d6d35e487217a489db2,/r/AesPleasingAsianGirls/comments/13qlj64/habin/,https://i.redd.it/29soeuf3nt1b1.jpg,13qlj64.jpg,data/image/13qlj64.jpg,SexyAsianDiffusion,True,False,False,[]
820,13qmsn4,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,a2189d7026d58082336ddcad3667161e,/r/AesPleasingAsianGirls/comments/13qmsn4/song...,https://i.redd.it/cdj3scvnvt1b1.jpg,13qmsn4.jpg,data/image/13qmsn4.jpg,SexyAsianDiffusion,True,False,False,[]


In [17]:
display(f"Total Number Of New Images - {new_df.shape[0]}")
display("0-1 Image Acquisition Complete - Writing new output and Shutting Down")

NameError: name 'new_df' is not defined